## Loading the required Libraries

In [49]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist, cifar10, cifar100
from keras.models import Sequential
from keras.applications import VGG19
from keras.applications import VGG16
from keras.layers import Dense, Flatten
from keras.layers import Dropout,Conv2D, MaxPooling2D,Activation
from keras.applications import resnet50,mobilenet
# from tensorflow.keras import Sequential
import numpy as np
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Loading the Dataset

In [50]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape

(60000, 28, 28)

## Normalizing and reshaping the data

In [51]:
img_rows, img_cols, channels = 28, 28, 1
num_classes = 10
x_train = x_train / 255
x_test = x_test / 255
x_train = x_train.reshape((-1, img_rows, img_cols, channels))
x_test = x_test.reshape((-1, img_rows, img_cols, channels))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print("Data shapes", x_test.shape, y_test.shape, x_train.shape, y_train.shape)

Data shapes (10000, 28, 28, 1) (10000, 10) (60000, 28, 28, 1) (60000, 10)


## Defining the functions for recall, F1score and Precision

In [52]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## BaseLine Model

In [53]:
def create_basemodel():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m,recall_m])
    return model

## Training the model and Saving the weights

In [55]:
base_model = create_basemodel()
history=base_model.fit(x_train, y_train,batch_size=258,epochs=10)
# base_model.save('base_model_nrml.h5')

Epoch 1/10
233/233 [==============================] - 8s 33ms/step - loss: 0.7900 - accuracy: 0.7537 - f1_m: 0.6908 - precision_m: 0.7908 - recall_m: 0.6387
Epoch 2/10
233/233 [==============================] - 8s 33ms/step - loss: 0.2742 - accuracy: 0.9182 - f1_m: 0.9177 - precision_m: 0.9366 - recall_m: 0.8995
Epoch 3/10
233/233 [==============================] - 8s 33ms/step - loss: 0.2082 - accuracy: 0.9384 - f1_m: 0.9389 - precision_m: 0.9519 - recall_m: 0.9263
Epoch 4/10
233/233 [==============================] - 8s 35ms/step - loss: 0.1716 - accuracy: 0.9491 - f1_m: 0.9497 - precision_m: 0.9587 - recall_m: 0.9410
Epoch 5/10
233/233 [==============================] - 8s 34ms/step - loss: 0.1531 - accuracy: 0.9541 - f1_m: 0.9550 - precision_m: 0.9631 - recall_m: 0.9472
Epoch 6/10
233/233 [==============================] - 8s 34ms/step - loss: 0.1365 - accuracy: 0.9588 - f1_m: 0.9592 - precision_m: 0.9663 - recall_m: 0.9522
Epoch 7/10
233/233 [==============================] - 8s 3

In [14]:
base_model.save(r'/content/drive/My Drive/saved_models_nrml/base_model_nrml.h5')

In [56]:
Test_Result=base_model.evaluate(x=x_test, y=y_test, verbose=0)
print("Analysis on normal Images")
print("Base Loss: ", Test_Result[0])
print("Accuracy :" , Test_Result[1])
print("F1_Score :", Test_Result[2])

Analysis on normal Images
Base Loss:  0.05534285306930542
Accuracy : 0.9829000234603882
F1_Score : 0.9826865792274475


## Defining functions for generating attacks

In [57]:
def adversarial_pattern(image, label,model):
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.MSE(label, prediction)
    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    return signed_grad

In [58]:
def generate_adversarials(batch_size,attack,x_train,y_train,model):
    while True:
        x = []
        y = []
        for batch in range(batch_size):
            N = random.randint(0, 100)
            label = y_train[N]
            image = x_train[N]
            perturbations = adversarial_pattern(image.reshape(-1,img_rows, img_cols, channels),label,model).numpy()
            epsilon = 0.1
            decay_factor=1.0
            iter = 10
            alpha = epsilon/iter
            g=0
            pert_out = image
            if attack == "fgsm":
              epsilon = 0.1
              pert_out = pert_out + perturbations * epsilon
              x.append(pert_out)
              y.append(y_train[N])
            elif attack == "sp":
              row,col,ch = image.shape
              s_vs_p = 0.5
              amount = 0.4
              out = np.copy(image)
              # Salt mode
              num_salt = np.ceil(amount * image.size * s_vs_p)
              # print(num_salt)
              for j in image.shape:
                  if j == 1 :
                      coords = np.random.randint(0, int(num_salt))
                  else:
                      coords = np.random.randint(0, j - 1, int(num_salt))
              # print(coords)
              out[coords[0]] = 1

              # Pepper mode
              num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
              # print (num_pepper)
              for j in image.shape:
                  if j == 1 :
                      coords = np.random.randint(0, int(num_salt))
                  else:
                      coords = np.random.randint(0, j - 1, int(num_salt))
              out[coords[0]] = 0
              # plt.imshow(out)
              # plt.show()
              print(out)
              x.append(out)
              y.append(train_y_final[N])
            elif attack == "rnoise":
              noise_factor = 1
              noisy = image + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=image.shape) 
              out = np.clip(noisy, 0., 1.)
              x.append(out)
              y.append(y_train[N])
            elif attack == "pgd":
              for i in range(iter-1):
                pert_out = pert_out + alpha*perturbations
                pert_out = torch.from_numpy(pert_out)
                pert_out = torch.clamp(pert_out, 0, 1)
                if torch.norm((pert_out-image),p=float('inf')) > epsilon:
                  break
                pert_out = pert_out.numpy()
              x.append(pert_out)
              y.append(y_train[N])
            elif attack == "mifgsm":
              for i in range(iter-1):
                p = torch.from_numpy(perturbations)
                g = decay_factor*g + p/torch.norm(p)
                pert_out = torch.from_numpy(pert_out)
                pert_out = pert_out + alpha*torch.sign(g)
                pert_out = torch.clamp(pert_out, 0, 1)
                if torch.norm((pert_out-image),p=float('inf')) > epsilon:
                  break
                pert_out = pert_out.numpy()
              x.append(pert_out)
              y.append(y_train[N])
        x = np.asarray(x).reshape((batch_size, img_rows, img_cols, channels))
        y = np.asarray(y)
        yield x, y

## Generating data with perturbated images using **FGSM** and evaluating the model's performance on the generated data

In [64]:
#Analysis before defense with adversarial inputs
img_rows, img_cols, channels = 28, 28, 1
x_adversarial_test,y_adversarial_test = next(generate_adversarials(10000,'fgsm',x_train,y_train,base_model))
Test_Result=base_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

[2.958076238632202, 0.275299996137619, 0.22927746176719666, 0.2472507357597351, 0.21425719559192657]


## Generating a data of perturbated images using **FGSM** to train the model and evaluating the model after training

In [65]:
img_rows, img_cols, channels = 28, 28, 1
x_adversarial_train, y_adversarial_train = next(generate_adversarials(15000,"fgsm",x_train,y_train,base_model))
fgsm_model = create_basemodel()
fgsm_model.load_weights(r'/content/drive/My Drive/saved_models_nrml/base_model_nrml.h5')
fgsm_model.fit(x_adversarial_train, y_adversarial_train,batch_size=256,epochs=10)
Test_Result=fgsm_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

Epoch 1/10
59/59 [==============================] - 2s 32ms/step - loss: 0.1306 - accuracy: 0.9629 - f1_m: 0.9630 - precision_m: 0.9680 - recall_m: 0.9583
Epoch 2/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0085 - accuracy: 0.9984 - f1_m: 0.9984 - precision_m: 0.9985 - recall_m: 0.9982
Epoch 3/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0046 - accuracy: 0.9991 - f1_m: 0.9991 - precision_m: 0.9992 - recall_m: 0.9991
Epoch 4/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0044 - accuracy: 0.9989 - f1_m: 0.9989 - precision_m: 0.9989 - recall_m: 0.9989
Epoch 5/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0026 - accuracy: 0.9994 - f1_m: 0.9994 - precision_m: 0.9995 - recall_m: 0.9994
Epoch 6/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0021 - accuracy: 0.9995 - f1_m: 0.9994 - precision_m: 0.9994 - recall_m: 0.9994
Epoch 7/10
59/59 [==============================] - 2s 33ms/step - los

## Generating data with perturbated images using **PGD** and evaluating the model's performance on the generated data

In [66]:
#Analysis before defense with adversarial inputs
img_rows, img_cols, channels = 28,28,1
# comparision=pd.DataFrame()
x_adversarial_test,y_adversarial_test = next(generate_adversarials(10000,'pgd',x_train,y_train,base_model))
Test_Result=base_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

[1.1061619520187378, 0.660099983215332, 0.6655484437942505, 0.6832377314567566, 0.6493610143661499]


## Generating a data of perturbated images using **PGD** to train the model and evaluating the model after training

In [68]:
img_rows, img_cols, channels = 28, 28, 1
x_adversarial_train, y_adversarial_train = next(generate_adversarials(15000,"pgd",x_train,y_train,base_model))
pgd_model = create_basemodel()
pgd_model.load_weights(r'/content/drive/My Drive/saved_models_nrml/base_model_nrml.h5')
pgd_model.fit(x_adversarial_train, y_adversarial_train,batch_size=256,epochs=10)
Test_Result=pgd_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

Epoch 1/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0584 - accuracy: 0.9848 - f1_m: 0.9842 - precision_m: 0.9872 - recall_m: 0.9814
Epoch 2/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0047 - accuracy: 0.9992 - f1_m: 0.9992 - precision_m: 0.9993 - recall_m: 0.9992
Epoch 3/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0032 - accuracy: 0.9992 - f1_m: 0.9992 - precision_m: 0.9992 - recall_m: 0.9992
Epoch 4/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0027 - accuracy: 0.9995 - f1_m: 0.9996 - precision_m: 0.9996 - recall_m: 0.9995
Epoch 5/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0019 - accuracy: 0.9998 - f1_m: 0.9998 - precision_m: 0.9998 - recall_m: 0.9997
Epoch 6/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0015 - accuracy: 0.9996 - f1_m: 0.9996 - precision_m: 0.9996 - recall_m: 0.9996
Epoch 7/10
59/59 [==============================] - 3s 57ms/step - los

## Generating data with perturbated images using **MIFGSM** and evaluating the model's performance on the generated data

In [69]:
#Analysis before defense with adversarial inputs
img_rows, img_cols, channels = 28,28,1
# comparision=pd.DataFrame()
x_adversarial_test,y_adversarial_test = next(generate_adversarials(10000,'mifgsm',x_train,y_train,base_model))
Test_Result=base_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

[1.1263490915298462, 0.6486999988555908, 0.6544972658157349, 0.6713201999664307, 0.6389776468276978]


## Generating a data of perturbated images using **MIFGSM** to train the model and evaluating the model after training

In [70]:
img_rows, img_cols, channels = 28, 28, 1
x_adversarial_train, y_adversarial_train = next(generate_adversarials(15000,"mifgsm",x_train,y_train,base_model))
mifgsm_model = create_basemodel()
mifgsm_model.load_weights(r'/content/drive/My Drive/saved_models_nrml/base_model_nrml.h5')
mifgsm_model.fit(x_adversarial_train, y_adversarial_train,batch_size=256,epochs=10)
Test_Result=mifgsm_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

Epoch 1/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0535 - accuracy: 0.9845 - f1_m: 0.9847 - precision_m: 0.9870 - recall_m: 0.9824
Epoch 2/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0048 - accuracy: 0.9993 - f1_m: 0.9992 - precision_m: 0.9993 - recall_m: 0.9992
Epoch 3/10
59/59 [==============================] - 2s 33ms/step - loss: 0.0026 - accuracy: 0.9997 - f1_m: 0.9997 - precision_m: 0.9997 - recall_m: 0.9997
Epoch 4/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0031 - accuracy: 0.9992 - f1_m: 0.9992 - precision_m: 0.9993 - recall_m: 0.9991
Epoch 5/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0018 - accuracy: 0.9997 - f1_m: 0.9997 - precision_m: 0.9997 - recall_m: 0.9997
Epoch 6/10
59/59 [==============================] - 2s 32ms/step - loss: 0.0013 - accuracy: 0.9998 - f1_m: 0.9998 - precision_m: 0.9998 - recall_m: 0.9997
Epoch 7/10
59/59 [==============================] - 2s 32ms/step - los

## Generating data with perturbated images using **Rnoise** and evaluating the model's performance on the generated data

In [71]:
#Analysis before defense with adversarial inputs
img_rows, img_cols, channels = 28,28,1
# comparision=pd.DataFrame()
x_adversarial_test,y_adversarial_test = next(generate_adversarials(10000,'rnoise',x_train,y_train,base_model))
Test_Result=base_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

[6.733301639556885, 0.1396999955177307, 0.134746715426445, 0.14641985297203064, 0.12509983777999878]


## Generating a data of perturbated images using **Rnoise** to train the model and evaluating the model after training

In [72]:
img_rows, img_cols, channels = 28, 28, 1
x_adversarial_train, y_adversarial_train = next(generate_adversarials(15000,"rnoise",x_train,y_train,base_model))
rnoise_model = create_basemodel()
rnoise_model.load_weights(r'/content/drive/My Drive/saved_models_nrml/base_model_nrml.h5')
rnoise_model.fit(x_adversarial_train, y_adversarial_train,batch_size=256,epochs=10)
Test_Result=rnoise_model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0)
print(Test_Result)

Epoch 1/10
59/59 [==============================] - 2s 32ms/step - loss: 2.1970 - accuracy: 0.3755 - f1_m: 0.3033 - precision_m: 0.5483 - recall_m: 0.2148
Epoch 2/10
59/59 [==============================] - 2s 32ms/step - loss: 1.2441 - accuracy: 0.5875 - f1_m: 0.5589 - precision_m: 0.7420 - recall_m: 0.4498
Epoch 3/10
59/59 [==============================] - 2s 32ms/step - loss: 0.9954 - accuracy: 0.6715 - f1_m: 0.6532 - precision_m: 0.7843 - recall_m: 0.5603
Epoch 4/10
59/59 [==============================] - 2s 33ms/step - loss: 0.8563 - accuracy: 0.7167 - f1_m: 0.7053 - precision_m: 0.8082 - recall_m: 0.6259
Epoch 5/10
59/59 [==============================] - 2s 35ms/step - loss: 0.7612 - accuracy: 0.7424 - f1_m: 0.7350 - precision_m: 0.8265 - recall_m: 0.6621
Epoch 6/10
59/59 [==============================] - 2s 33ms/step - loss: 0.6857 - accuracy: 0.7715 - f1_m: 0.7665 - precision_m: 0.8472 - recall_m: 0.7003
Epoch 7/10
59/59 [==============================] - 2s 33ms/step - los